In [1]:
from Polynomial import Polynomial, poly_round, PolynomialTensor
import numpy as np
from dataclasses import dataclass

In [2]:
SecretKey = Polynomial
PrivateKey = tuple[Polynomial, Polynomial]
RelinearizationKey = tuple[Polynomial, Polynomial]

In [3]:
### RLK V1
@dataclass
class BfvConfig:
    poly_len: int
    modulus: int
    base: int


def decompose_len(modulus: int, base: int) -> int:
    return int(np.floor(np.emath.logn(base, modulus))) + 1


def to_base(poly: Polynomial, base: int) -> np.ndarray:
    l = decompose_len(poly.modulus, base)

    def _to_base(arr: np.ndarray, base: int, index=1) -> list:
        if index >= l:
            return arr
        return [arr[-1] % base] + _to_base([arr[-1] // base], base, index + 1)

    return np.asarray(_to_base(poly.poly_mat, base))

class BfvMessage:

    def __init__(self, config: BfvConfig, rlk: RelinearizationKey,
                 u: Polynomial, v: Polynomial):
        self.config = config
        self.u = u
        self.v = v
        self.rlk = rlk

    def __add__(self, other: "BfvMessage") -> "BfvMessage":
        assert self.config == other.config
        return BfvMessage(self.config,
                          self.rlk,
                          u=(self.u + other.u) % self.config.modulus,
                          v=(self.v + other.v) % self.config.modulus)

    def __mul__(self, other: "BfvMessage") -> "BfvMessage":
        assert self.config == other.config

        def round_with_poly(poly: Polynomial) -> Polynomial:
            return Polynomial((np.round(poly.poly_mat)) % self.config.modulus,
                              self.config.modulus)

        v = round_with_poly((2 / self.config.modulus) * (self.v @ other.v))

        u = round_with_poly(
            (2 / self.config.modulus) * (self.v @ other.u + self.u @ other.v))

        uv = round_with_poly((2 / self.config.modulus) * (self.u @ other.u))

        uv_rlk = list(zip(to_base(uv, self.config.base), self.rlk))

        uvu = Polynomial(
            np.sum(np.asarray([(Polynomial(np.asarray(
                [uv]), self.config.modulus) @ rlk[0]).poly_mat
                               for uv, rlk in uv_rlk]),
                   axis=0), self.config.modulus)

        uvv = Polynomial(
            np.sum(np.asarray([(Polynomial(np.asarray(
                [uv]), self.config.modulus) @ rlk[1]).poly_mat
                               for uv, rlk in uv_rlk]),
                   axis=0), self.config.modulus)

        return BfvMessage(self.config,
                          self.rlk,
                          u=(u + uvu) % self.config.modulus,
                          v=(v + uvv) % self.config.modulus)

class BFV:

    @staticmethod
    def keygen(
            conf: BfvConfig
    ) -> tuple[SecretKey, PrivateKey, RelinearizationKey]:
        s = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
        e = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
        A = Polynomial.random_polynomial(conf.poly_len, conf.modulus)

        l = decompose_len(conf.modulus, conf.base)
        rlk = []

        for i in range(l):
            rA = Polynomial.random_polynomial(conf.poly_len, conf.modulus)
            re = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0,
                                              1)
            rb = (-1 * (rA @ s + re) + (conf.base**i * (s @ s))) % conf.modulus
            rlk.append((rA, rb))

        return (s, (A, (-1 * (A @ s + e)) % conf.modulus), rlk)

    @staticmethod
    def encrypt(conf: BfvConfig, pk: PrivateKey, rlk: RelinearizationKey,
                message: list) -> BfvMessage:
        assert isinstance(message, list)
        A, b = pk
        e1 = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
        e2 = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
        r = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 2)

        v = (b @ r + e1 + Polynomial(
            np.asarray([message]) *
            (conf.modulus // 2), conf.modulus)) % conf.modulus
        u = (A @ r + e2) % conf.modulus

        return BfvMessage(conf, rlk, u=u, v=v)

    def decrypt(sk: SecretKey, m_enc: BfvMessage) -> list:
        return (np.round(
            ((2 / m_enc.config.modulus) *
             ((m_enc.v + m_enc.u @ sk) % m_enc.config.modulus)).poly_mat) %
                2).tolist()[0]


In [4]:
#### RLK V2

# @dataclass
# class BfvConfig:
#     poly_len: int
#     modulus: int
#     p: int


# class BfvMessage:

#     def __init__(self, config: BfvConfig, rlk: RelinearizationKey,
#                  u: Polynomial, v: Polynomial):
#         self.config = config
#         self.u = u
#         self.v = v
#         self.rlk = rlk

#     def __add__(self, other: "BfvMessage") -> "BfvMessage":
#         assert self.config == other.config
#         return BfvMessage(self.config,
#                           self.rlk,
#                           u=(self.u + other.u) % self.config.modulus,
#                           v=(self.v + other.v) % self.config.modulus)

#     def __mul__(self, other: "BfvMessage") -> "BfvMessage":
#         assert self.config == other.config

#         def round_with_poly(poly: Polynomial) -> Polynomial:
#             return Polynomial((np.round(poly.poly_mat)) % self.config.modulus,
#                               self.config.modulus)

#         v = round_with_poly((2 / self.config.modulus) * (self.v @ other.v))

#         u = round_with_poly(
#             (2 / self.config.modulus) * (self.v @ other.u + self.u @ other.v))

#         uv = round_with_poly(
#             (2 / self.config.modulus) * (self.u @ other.u)).change_modulus(
#                 self.config.p * self.config.modulus)
#         uvu = round_with_poly((uv @ self.rlk[0]) * (1 / self.config.p))
#         uvv = round_with_poly((uv @ self.rlk[1]) * (1 / self.config.p))

#         return BfvMessage(self.config,
#                           self.rlk,
#                           u=(u + uvu) % self.config.modulus,
#                           v=(v + uvv) % self.config.modulus)


# class BFV:

#     @staticmethod
#     def keygen(
#             conf: BfvConfig
#     ) -> tuple[SecretKey, PrivateKey, RelinearizationKey]:
#         s = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
#         e = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
#         A = Polynomial.random_polynomial(conf.poly_len, conf.modulus)

#         rs = Polynomial(s.poly_mat, conf.p * conf.modulus)
#         re = Polynomial.random_polynomial(conf.poly_len, conf.p * conf.modulus,
#                                           0, 1)
#         rA = Polynomial.random_polynomial(conf.poly_len, conf.p * conf.modulus)

#         return (s, (A, (-1 * (A @ s + e)) % conf.modulus),
#                 (rA, (-1 * (rA @ rs + re) + conf.p * (rs @ rs)) %
#                  (conf.p * conf.modulus)))

#     @staticmethod
#     def encrypt(conf: BfvConfig, pk: PrivateKey, rlk: RelinearizationKey,
#                 message: list) -> BfvMessage:
#         assert isinstance(message, list)
#         A, b = pk
#         e1 = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
#         e2 = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
#         r = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 2)

#         v = (b @ r + e1 + Polynomial(
#             np.asarray([message]) *
#             (conf.modulus // 2), conf.modulus)) % conf.modulus
#         u = (A @ r + e2) % conf.modulus

#         return BfvMessage(conf, rlk, u=u, v=v)

#     def decrypt(sk: SecretKey, m_enc: BfvMessage) -> list:
#         return (np.round(
#             ((2 / m_enc.config.modulus) *
#              ((m_enc.v + m_enc.u @ sk) % m_enc.config.modulus)).poly_mat) %
#                 2).tolist()[0]


In [5]:
conf = BfvConfig(10, 1000, 1000**3)
sk, pk, rlk = BFV.keygen(conf)

In [82]:
# conf = BfvConfig(8, 1000, 1000**3)
conf = BfvConfig(8, 1000, 10)
sk, pk, rlk = BFV.keygen(conf)
m1 = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
m_e1 = BFV.encrypt(conf, pk, rlk, m1.poly_mat[0].tolist())
for i in range(10000):
    m2 = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
    m_e2 = BFV.encrypt(conf, pk, rlk, m2.poly_mat[0].tolist())
    m_e1 = m_e1*m_e2
    m1 = (m1 @ m2)%2
    assert BFV.decrypt(sk, m_e1) == m1.poly_mat[0].tolist(), f"{i}: {m1} -- {BFV.decrypt(sk, m_e1)}"


AssertionError: 1: 0.0 + 0.0·x + 1.0·x² + 1.0·x³ + 1.0·x⁴ + 1.0·x⁵ + 0.0·x⁶ + 0.0·x⁷ -- [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]

In [37]:
conf = BfvConfig(8, 100000, 10)
sk, pk, rlk = BFV.keygen(conf)

In [38]:
m1 = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
m_e1 = BFV.encrypt(conf, pk, rlk, m1.poly_mat[0].tolist())
m2 = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
m_e2 = BFV.encrypt(conf, pk, rlk, m2.poly_mat[0].tolist())

In [39]:
%timeit m_e1 * m_e2

909 µs ± 37.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
